In [1]:
from simulated_annealing import *

In [2]:
def calibrate_parameter_all():
    solutions_with_params = []
    for termination_condition in [10**x for x in range(4)]:
        for halting_condition in [10**x for x in range(4)]:
            for T_max in [10**x for x in range(5)]:
                for r in range(10, 100, 10):
                    print(f'params: T_max={T_max}, r={r/100}, termination_condition={termination_condition}, halting_condition={halting_condition}')
                    solutions = test_SA(T_max, r/100, termination_condition, halting_condition, False)
                    solutions_with_params.append([
                        (T_max, r, termination_condition, halting_condition),
                        solutions
                        ])
    return solutions_with_params

In [3]:
def calibrate_parameter_per_example(example):
    for termination_condition in [10**x for x in range(4)]:
        for halting_condition in [10**x for x in range(4)]:
            for T_max in [10**x for x in range(5)]:
                for r in range(10, 100, 10):
                    print(f'params: T_max={T_max}, r={r/100}, termination_condition={termination_condition}, halting_condition={halting_condition}')
                    params = T_max, r, termination_condition, halting_condition, True
                    test_SA_per_example(example, *params)
                    

In [ ]:
calibrate_parameter_per_example(get_examples()[0])

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_inst

Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not sati

Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not sati

Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not sati

Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 20
Last solution: 20
passed

rws_instances/Example1.txt
Not sati

Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not sati

Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 40
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 20
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 40
passed

rws_instances/Example1.txt
Not sati

Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 80
Last solution: 80
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 80
Last solution: 80
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 80
Last solution: 80
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 60
Last solution: 60
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 80
Last solution: 80
passed

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 80
Last solution: 80
passed

rws_instances/Example1.txt
Not sati